In [ ]:
class Power():
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent

class Number():
    def __init__(self,number):
        self.number = number

class Variable():
    def __init__(self,symbol):
        self.symbol = symbol

Power(Variable("x"),Number(2))
##model of the expression x^2

class Product():
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2